In [ ]:
swid = ""
espn_s2 = ""
league_id = ""
season = 2022
n_active_players = 8
week = 1
scoring_period = 7
home_team = ""
away_team = ""
start_date = ""
end_date = ""
use_current_score = False
stat_type_code = "season average"

In [ ]:
STAT_PERIOD_DICT = {
    'season average': '002022',
    'last 7 days average': '012022',
    'last 15 days average': '022022',
    'last 30 days average': '032022',
    "season's projections": '102022',
    'previous season average': '002021'
}

In [ ]:
import sys
import pandas as pd
sys.path.append('../')
from src.EspnFantasyMatchUp import EspnFantasyMatchUp
from src.utils.io_json import read_json

# App description

This app prepares for an upcoming match-up between two fantasy teams. This is based on a league with *Head to Head Each Category* scoring type and 8 stat categories (FG%, FT%, 3PM, REB, AST, STL, BLK, TO, PTS).

* It compares their schedule (number of starter players and unused players) 
* It compares the teams' historic stats up to this round
* Simulates/projects the match-up based on the players' stats and schedule. 
* Allows for scenarios, such as replace player X with player Y, etc. (feature not currently available, will be added soon)

## App parameters

* `swid`: If your league is public, leave blank. Otherwise: In Safari these cookies can be found by opening the Storage tab of Developer tools (developer tools can be turned on in preferences), and looking under espn.com in the Cookies folder. In Chrome, they can be found in Settings -> Cookies and other site data -> See all cookies and site data, and search for ESPN.
* `espn_s2`: If your league is public, leave blank. Otherwise, follow above instructions. **CAUTION:** It is likely the cookie id to contain the `%` character. The corresponding field does not accept such a character. Replace it with a `!` (exclamation mark)
* `League ID`: Go to your ESPN league page. The URL contains `leagueId=123455`. Copy the number next to the `leagueid` parameters
* `Season`: Leave this to current season, i.e. 2022, unless you want to run analysis of a round of a previous season
* `Starters`: most leagues have 8 startersy. Change this accordingly.
* `Round`: The round for analysis. It must be a future round, or a round in progress.
* `Home team`: Abbrebiation of the home team
* `Away team`: Abbreviation of the away team
* `Start date`: The start date of the analysis. If the round hasn't started, it should the start date of the round. If the round is in progress, it should be the current date. The format must be "YYYY-MM-DD"
* `End date`: The end data of the round. It must be later than the start date. The format must be "YYYY-MM-DD"
* `Use rounds stats`: If the round is in progress, use the accumulated stats so far to make the projection. I.e., if True, the projection will be the combination of the current stats and the projection stats for the remaining days. If False, the projection will be based only on the simulation results.
* `Stat type`: The type of stats for the simulation, i.e. "Season" (average season stats), "Last 30" (average of the last 30 days stats), etc.

In [ ]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [ ]:
cookies = {
    "swid": swid,
    "espn_s2": espn_s2.replace("!", "%")
}
league_settings = {
    "league_id": int(league_id),
    "season": season,
    "n_active_players": n_active_players
}

In [ ]:
espn = EspnFantasyMatchUp(
    cookies, league_settings,
    week, 
    home_team=home_team, away_team=away_team, 
    start_date=start_date, end_date=end_date,
    stat_type_code=STAT_PERIOD_DICT[stat_type_code]
)

# Head to head comparison of season historic stats

In [ ]:
espn.h2h_season_stats_comparison()

# Compare Schedule (players per day)

In [ ]:
espn.compare_schedules()

# Simulate Schedule

In [ ]:
espn.simulation(use_current_score=use_current_score)

# Scenarios

Add or remove players from the roster of either the home or the away team

```
home_scn_players={
    "add": {
        <plyaer_name>: [<date1>, <date2>],
        <player_name2>: [<date1>]
    }, 
    "remove": {
        <plyaer_name3>: [<date1>, <date2>],
        <player_name4>: [<date1>]
    }, 
}
```
Similar of the away players, using the `away_scn_players` argument. 

Leave date list empyt if you want to include all dates in the schedule between `start_date` and `end_date`

In [ ]:
# scn = EspnFantasyMatchUp(
#     week, home_team=home_team, away_team=away_team, start_date=pd.to_datetime("2022-02-16"), end_date=end_date,
# #     home_scn_players={"add": {"Eric Gordon": []}, "remove": {"Nicolas Batum": []}}, 
#     stat_type_code="022022"
# )

In [ ]:
# scn.simulation(use_current_score=True)